In [2]:
import zipfile
import os
from io import BytesIO
from datetime import datetime

import polars as pl
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', None)

In [3]:
BASE_URL = os.environ.get("BASE_URL")
print(BASE_URL)

/home/prueba/cgr_corrupcion/


In [4]:
# datasets
folder_data = BASE_URL + 'data/'

datasets = {
    'ds_airhsp' : 'DS01_AIRHSP.zip',
    'ds_seace' : 'DS02_SEACE.zip',
    'ds_siaf' : 'DS03_ORDENES_SIAF.zip',
    'ds_empresas' : 'DS04_REPRESENTANTES_EMPRESAS.zip',
    'ds_servir' : 'DS05_SANCIONADOS_SERVIR.zip',
    'ds_gstos' : 'DS06_GASTOS_EJECUTORA.zip',
    'ds_proveedor' : 'DS07_PROVEEDORES_PROYECTOS.zip',
    'ds_proyectos' : 'DS07_PROYECTOS_INVERSION.zip',
    'ds_inco' : 'DS08_INCO.zip'
}


Tamaños de archivos CSV en DS01_AIRHSP.zip:
DS01_AIRHSP/AIRHSP_2201.csv: 0.75 GB
DS01_AIRHSP/AIRHSP_2202.csv: 0.75 GB
DS01_AIRHSP/AIRHSP_2203.csv: 0.75 GB
DS01_AIRHSP/AIRHSP_2204.csv: 0.76 GB
DS01_AIRHSP/AIRHSP_2205.csv: 0.76 GB
DS01_AIRHSP/AIRHSP_2206.csv: 0.76 GB
DS01_AIRHSP/AIRHSP_2207.csv: 0.77 GB
DS01_AIRHSP/AIRHSP_2208.csv: 0.77 GB
DS01_AIRHSP/AIRHSP_2209.csv: 0.77 GB
DS01_AIRHSP/AIRHSP_2210.csv: 0.77 GB
DS01_AIRHSP/AIRHSP_2211.csv: 0.78 GB
DS01_AIRHSP/AIRHSP_2212.csv: 0.78 GB

Tamaños de archivos CSV en DS02_SEACE.zip:
DS02_SEACE.csv: 0.06 GB

Tamaños de archivos CSV en DS03_ORDENES_SIAF.zip:
DS03_ORDENES_SIAF/DS03_ORDENESSIAF_2020.csv: 4.16 GB
DS03_ORDENES_SIAF/DS03_ORDENESSIAF_2021.csv: 4.82 GB
DS03_ORDENES_SIAF/DS03_ORDENESSIAF_2022.csv: 5.40 GB

Tamaños de archivos CSV en DS04_REPRESENTANTES_EMPRESAS.zip:
DS04_REPRESENTANTES.csv: 3.66 GB

Tamaños de archivos CSV en DS05_SANCIONADOS_SERVIR.zip:
DS05_SERVIR_SANCIONADOS.csv: 0.01 GB

Tamaños de archivos CSV en DS06_GASTOS_EJEC

In [5]:
def get_dataset(id_dataset,id_elem):
    archivo_zip = datasets[id_dataset]
    ruta_zip = os.path.join(folder_data, archivo_zip)
    df = pl.DataFrame()
    
    with zipfile.ZipFile(ruta_zip, 'r') as zip_ref:
        print(f"\nTamaños de archivos CSV en {archivo_zip}:")
        # Iterar sobre los archivos dentro del zip
        list_archivo = []
        for nombre_archivo in zip_ref.namelist():
            # Verificar si el archivo es un archivo CSV
            
            if nombre_archivo.endswith('.csv'):
                list_archivo.append(nombre_archivo)
            
        print(list_archivo)
        print(list_archivo[id_elem])
        with zip_ref.open(list_archivo[id_elem]) as archivo_csv:
            # df = pl.read_csv(archivo_csv, ignore_errors=True)
            df = pd.read_csv(archivo_csv)
            
        # Obtener el tamaño del archivo en bytes
        # tamano_bytes = zip_ref.getinfo(nombre_archivo).file_size

        # # Convertir el tamaño a megabytes para mayor legibilidad
        # tamano_mb = tamano_bytes / (1024 ** 3)

        # # Imprimir el nombre del archivo y su tamaño
        # print(f'{nombre_archivo}: {tamano_mb:.2f} GB')
            # break
    return df

In [6]:
def get_dataset_multiple_files(id_dataset):

    archivo_zip = datasets[id_dataset]
    ruta_zip = os.path.join(folder_data, archivo_zip)
    
    # Crear un DataFrame vacío
    df = pd.DataFrame()
    
    with zipfile.ZipFile(ruta_zip, 'r') as zip_ref:
        # Iterar sobre los nombres de los archivos en el zip
        for nombre_archivo in zip_ref.namelist():
            # Verificar si el archivo es un archivo CSV
            if nombre_archivo.endswith('.csv'):
                # Leer el contenido del archivo CSV
                with zip_ref.open(nombre_archivo) as archivo_csv:
                    # contenido_csv = archivo_csv.read().decode('utf-8')
                    
                    # # Utilizar BytesIO para convertir el contenido en un objeto de bytes
                    # bytes_io = BytesIO(contenido_csv)
                    
                    # Leer el DataFrame desde el objeto de bytes
                    df_temp = pd.read_csv(archivo_csv)
                    
                    # Agregar el DataFrame al DataFrame principal
                    df = pd.concat([df, df_temp])
    
    # Imprimir el DataFrame resultante
    return df

### AIRHSP

In [6]:
%%time

airhsp_data = get_dataset_multiple_files("ds_airhsp")

CPU times: user 4min 18s, sys: 41.1 s, total: 5min
Wall time: 5min 4s


In [108]:
len(airhsp_data["CENT_RUC"].unique())

892

In [7]:
airhsp_data.head()

,PERIODO,NTDI_CODIGO,CTDI_DESCRIPCION,NUMERO_DOCUMENTO_A,FECHA_INGRESO,FECHA_CESE,DESC_GRUPO_OCUPACIONAL,DESC_CARGO_ESTRUCTURAL,DESC_CARGO_FUNCIONAL,CENT_RUC,CENT_NOMBRE,OBANT_NIVEL_GOB,OBANT_SECTOR,OBANT_GRUPO_FUNCIONAL,NOMBRE_DEPARTAMENTO,NOMBRE_PROVINCIA,NOMBRE_DISTRITO,CODIGO_ESTABLECIMIENTO,DESC_ESTABLECIMIENTO,TIPO_REGISTRO,DESC_TIPO_REGISTRO,COSTO_PRESUP,INGRESO_IMPONIBLE,CARGA_SOCIAL,INGRESO_NO_IMPONIBLE,INCENTIVO_UNICO,GUARDIA_HOSPITALARIA,ESCOLARIDAD,AGUINALDO,OTROS_OCASIONALES,TOTAL_MENSUAL,TOTAL_OCASIONAL,TOTAL_ANUAL
0,2022-01,1,DNI,10000482,31/12/1999,NaN,Profesorado,Profesor V-40 con Título Pedagógico,No tiene,20393274655,UNIDAD DE GESTIÓN EDUCATIVA LOCAL CORONEL PORT...,GOBIERNO REGIONAL,GOBIERNOS REGIONALES,GOBIERNO REGIONAL,UCAYALI,CORONEL PORTILLO,CALLARIA,70632,EDUCACION CORONEL PORTILLO,2,Pensionistas,1267.68,519.96,0.00,664.39,0.0,0.00,400.0,600.0,0.0,1184.35,1000.0,15212.20
1,2022-01,1,DNI,10002552,02/01/2004,NaN,Profesionales,Servidor Profesional E,CONTADOR,20195970751,DIRECCIÓN REGIONAL DE EDUCACIÓN UCAYALI,GOBIERNO REGIONAL,GOBIERNOS REGIONALES,GOBIERNO REGIONAL,UCAYALI,CORONEL PORTILLO,CALLARIA,359601,ESUA - EDUARDO MEZA SARAVIA,1,Activos,2262.97,598.98,83.70,350.66,1230.0,0.00,400.0,600.0,0.0,2263.34,1000.0,28160.08
2,2022-01,1,DNI,10005000,01/09/1984,NaN,Tecnicos,Servidor Tecnico B,TECNICO EN ENFERMERIA I,20198261476,DIRECCIÓN REGIONAL DE SALUD UCAYALI,GOBIERNO REGIONAL,GOBIERNOS REGIONALES,GOBIERNO REGIONAL,UCAYALI,CORONEL PORTILLO,CALLARIA,428678,P.S. I-2 SHIRAMBARI.,1,Activos,3211.88,1382.55,124.43,1312.45,0.0,433.55,400.0,600.0,0.0,3252.98,1000.0,40035.76
3,2022-01,1,DNI,10008979,14/03/2019,NaN,Reforma Magisterial,Profesor I-30 con Título Pedagógico,PROFESOR,20393274655,UNIDAD DE GESTIÓN EDUCATIVA LOCAL CORONEL PORT...,GOBIERNO REGIONAL,GOBIERNOS REGIONALES,GOBIERNO REGIONAL,UCAYALI,CORONEL PORTILLO,CALLARIA,401196,EBRS - COMERCIO 64,1,Activos,2583.63,1560.20,140.42,940.10,0.0,0.00,400.0,600.0,0.0,2740.72,1000.0,32688.64
4,2022-01,1,DNI,10011497,NaN,NaN,Sin Grupo Ocupacional,No tiene,No tiene,20254165035,OFICINA DE NORMALIZACIÓN PREVISIONAL - ONP,GOBIERNO NACIONAL,ECONOMÍA Y FINANZAS,ECONOMICA PRODUCTIVA,LIMA,LIMA,LIMA,60036,ONP - HUANUCO,3,Sobrevivientes,385.00,0.00,0.00,385.00,0.0,0.00,0.0,0.0,0.0,385.00,0.0,4620.00


In [8]:
airhsp_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27641182 entries, 0 to 2355409
Data columns (total 33 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   PERIODO                 object 
 1   NTDI_CODIGO             int64  
 2   CTDI_DESCRIPCION        object 
 3   NUMERO_DOCUMENTO_A      int64  
 4   FECHA_INGRESO           object 
 5   FECHA_CESE              object 
 6   DESC_GRUPO_OCUPACIONAL  object 
 7   DESC_CARGO_ESTRUCTURAL  object 
 8   DESC_CARGO_FUNCIONAL    object 
 9   CENT_RUC                int64  
 10  CENT_NOMBRE             object 
 11  OBANT_NIVEL_GOB         object 
 12  OBANT_SECTOR            object 
 13  OBANT_GRUPO_FUNCIONAL   object 
 14  NOMBRE_DEPARTAMENTO     object 
 15  NOMBRE_PROVINCIA        object 
 16  NOMBRE_DISTRITO         object 
 17  CODIGO_ESTABLECIMIENTO  int64  
 18  DESC_ESTABLECIMIENTO    object 
 19  TIPO_REGISTRO           int64  
 20  DESC_TIPO_REGISTRO      object 
 21  COSTO_PRESUP            float64
 22

In [60]:
airhsp_data["FECHA_CESE"].unique()

array([nan, '31/01/2022', '31/12/2021', ..., '04/01/2012', '18/04/2018',
       '11/11/2018'], dtype=object)

In [252]:
airhsp_data["DESC_TIPO_REGISTRO"].unique()

array(['Pensionistas', 'Activos', 'Sobrevivientes',
       'Contrato Administrativo de Servicios', 'CPMP',
       'Modalidad Formativa', 'Convenio de Administración',
       'Palmas Magisteriales', 'Promotoras'], dtype=object)

In [253]:
airhsp_data["PERIODO"].unique()

array(['2022-01', '2022-02', '2022-03', '2022-04', '2022-05', '2022-06',
       '2022-07', '2022-08', '2022-09', '2022-10', '2022-11', '2022-12'],
      dtype=object)

In [29]:
airhsp_data[~airhsp_data["FECHA_CESE"].isna()].head()

,PERIODO,NTDI_CODIGO,CTDI_DESCRIPCION,NUMERO_DOCUMENTO_A,FECHA_INGRESO,FECHA_CESE,DESC_GRUPO_OCUPACIONAL,DESC_CARGO_ESTRUCTURAL,DESC_CARGO_FUNCIONAL,CENT_RUC,CENT_NOMBRE,OBANT_NIVEL_GOB,OBANT_SECTOR,OBANT_GRUPO_FUNCIONAL,NOMBRE_DEPARTAMENTO,NOMBRE_PROVINCIA,NOMBRE_DISTRITO,CODIGO_ESTABLECIMIENTO,DESC_ESTABLECIMIENTO,TIPO_REGISTRO,DESC_TIPO_REGISTRO,COSTO_PRESUP,INGRESO_IMPONIBLE,CARGA_SOCIAL,INGRESO_NO_IMPONIBLE,INCENTIVO_UNICO,GUARDIA_HOSPITALARIA,ESCOLARIDAD,AGUINALDO,OTROS_OCASIONALES,TOTAL_MENSUAL,TOTAL_OCASIONAL,TOTAL_ANUAL
1216,2022-01,1,DNI,18881475,13/07/2015,31/01/2022,Tecnicos,No tiene,ASISTENTE ADMINISTRATIVO,20530099645,MINISTERIO PUBLICO- GERENCIA ADMNINISTRATIVA D...,GOBIERNO NACIONAL,MINISTERIO PÚBLICO,ORG. CONST. AUTONOMO,PIURA,PIURA,PIURA,71373,MINISTERIO PUBLICO- GERENCIA ADMINISTRATIVA DE...,4,Contrato Administrativo de Servicios,1350.00,1300.00,108.00,0.00,0.0,0.0,0.0,600.0,0.0,1408.00,600.0,17496.00
1590,2022-01,1,DNI,27825389,01/01/2021,31/12/2021,Reforma Magisterial,Profesor V-30 con Título Pedagógico,PROFESOR,20316055631,UNIDAD DE GESTIÓN EDUCATIVA LOCAL SULLANA - UG...,GOBIERNO REGIONAL,GOBIERNOS REGIONALES,GOBIERNO REGIONAL,PIURA,SULLANA,SULLANA,355202,EBRP - 14794 MARIA INMAC. CONCEPCION,1,Activos,3683.78,2340.29,210.63,1260.16,0.0,0.0,400.0,600.0,0.0,3811.08,1000.0,46732.96
1795,2022-01,1,DNI,36790679,06/06/1984,11/06/2021,Reforma Magisterial,Profesor I-30 con Título Pedagógico,PROFESOR,20364513276,UNIDAD DE GESTIÓN EDUCATIVA LOCAL 13 YAUYOS - ...,GOBIERNO REGIONAL,GOBIERNOS REGIONALES,GOBIERNO REGIONAL,LIMA,YAUYOS,YAUYOS,375238,EBRP - 20730 JUAN DE LA MATA MICHUY R.,1,Activos,2983.63,1560.20,140.42,1340.10,0.0,0.0,400.0,600.0,0.0,3540.72,1000.0,37488.64
1796,2022-01,1,DNI,36790679,06/06/1984,31/12/2012,Reforma Magisterial,Profesor I-30 con Título Pedagógico,PROFESOR,20364513276,UNIDAD DE GESTIÓN EDUCATIVA LOCAL 13 YAUYOS - ...,GOBIERNO REGIONAL,GOBIERNOS REGIONALES,GOBIERNO REGIONAL,LIMA,YAUYOS,YAUYOS,375238,EBRP - 20730 JUAN DE LA MATA MICHUY R.,1,Activos,2983.63,1560.20,140.42,1340.10,0.0,0.0,400.0,600.0,0.0,3540.72,1000.0,37488.64
2389,2022-01,1,DNI,11073816,30/04/1991,31/12/2012,Reforma Magisterial,Profesor IV-30 con Título Pedagógico,PROFESOR,20331166830,UNIDAD DE GESTIÓN EDUCATIVA LOCAL 05,GOBIERNO NACIONAL,EDUCACIÓN,SOCIAL Y CULTURA,LIMA,LIMA,SAN JUAN DE LURIGANCHO,335605,EBRP - 114 VIRGEN DE CHAPI,1,Activos,3203.72,2028.25,182.54,1092.14,0.0,0.0,400.0,600.0,0.0,3302.93,1000.0,40635.16


##### TRABAJADORES

In [8]:
master_workers = pd.DataFrame()

In [346]:
%%time
# employee_data = airhsp_data[(airhsp_data["NUMERO_DOCUMENTO_A"]==10000482) | (airhsp_data["NUMERO_DOCUMENTO_A"]==18881475)]
employee_data = airhsp_data.copy()

In [270]:
# employee_data.iloc[9429,:]
# employee_data["FECHA_INGRESO"] = employee_data["FECHA_INGRESO"].apply(lambda x: "15/11/2015" if x =="15/11/5015" else x)
# employee_data["FECHA_INGRESO"] = employee_data["FECHA_INGRESO"].apply(lambda x: "02/01/2021" if x =="02/01/202" else x)

In [271]:
# %%time
# employee_data["FECHA_INGRESO"] = pd.to_datetime(employee_data['FECHA_INGRESO'], dayfirst=True)
# employee_data["FECHA_CESE"] = pd.to_datetime(employee_data['FECHA_CESE'], dayfirst=True)
# employee_data.loc[:,"FECHA_INGRESO"] = pd.to_datetime(employee_data["FECHA_INGRESO"], format='%d/%m/%Y')

In [243]:
# # Obtener la fecha de hoy
# date_today = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)

# # Reemplazar los valores nulos con la fecha de hoy
# employee_data['FECHA_CESE'].fillna(date_today, inplace=True)

In [9]:
%%time

airhsp_data = airhsp_data.drop_duplicates().reset_index(drop=True) # 2166

CPU times: user 3min 48s, sys: 30 s, total: 4min 18s
Wall time: 4min 21s


In [10]:
airhsp_data["PERIODO"] = pd.to_datetime(airhsp_data['PERIODO'], format='%Y-%m')

In [11]:
airhsp_data["PERIODO"].unique()

<DatetimeArray>
['2022-01-01 00:00:00', '2022-02-01 00:00:00', '2022-03-01 00:00:00',
 '2022-04-01 00:00:00', '2022-05-01 00:00:00', '2022-06-01 00:00:00',
 '2022-07-01 00:00:00', '2022-08-01 00:00:00', '2022-09-01 00:00:00',
 '2022-10-01 00:00:00', '2022-11-01 00:00:00', '2022-12-01 00:00:00']
Length: 12, dtype: datetime64[ns]

In [12]:
airhsp_data.tail(2)

,PERIODO,NTDI_CODIGO,CTDI_DESCRIPCION,NUMERO_DOCUMENTO_A,FECHA_INGRESO,FECHA_CESE,DESC_GRUPO_OCUPACIONAL,DESC_CARGO_ESTRUCTURAL,DESC_CARGO_FUNCIONAL,CENT_RUC,CENT_NOMBRE,OBANT_NIVEL_GOB,OBANT_SECTOR,OBANT_GRUPO_FUNCIONAL,NOMBRE_DEPARTAMENTO,NOMBRE_PROVINCIA,NOMBRE_DISTRITO,CODIGO_ESTABLECIMIENTO,DESC_ESTABLECIMIENTO,TIPO_REGISTRO,DESC_TIPO_REGISTRO,COSTO_PRESUP,INGRESO_IMPONIBLE,CARGA_SOCIAL,INGRESO_NO_IMPONIBLE,INCENTIVO_UNICO,GUARDIA_HOSPITALARIA,ESCOLARIDAD,AGUINALDO,OTROS_OCASIONALES,TOTAL_MENSUAL,TOTAL_OCASIONAL,TOTAL_ANUAL
27639014,2022-12-01,1,DNI,14251973,31/12/2015,NaN,Tecnicos,Servidor Tecnico C,TECNICO EN ENFERMERIA I,20602251641,DIRECCION DE REDES INTEGRADAS DE SALUD LIMA SUR,GOBIERNO NACIONAL,SALUD,SOCIAL Y CULTURA,LIMA,LIMA,VILLA EL SALVADOR,423863,C.S. I-4 JOSE GALVEZ,1,Activos,3118.33,1603.55,161.61,1431.45,0.0,0.0,400.0,600.0,0.0,3196.61,1000.0,39359.32
27639015,2022-12-01,1,DNI,13290525,24/04/2016,NaN,Profesionales de la Salud,No tiene,ENFERMERA/O,20602251641,DIRECCION DE REDES INTEGRADAS DE SALUD LIMA SUR,GOBIERNO NACIONAL,SALUD,SOCIAL Y CULTURA,LIMA,LIMA,VILLA EL SALVADOR,416059,UE 1685 REDES INTEGRADAS DE SALUD LIMA SUR,4,Contrato Administrativo de Servicios,2289.00,2239.00,186.30,0.00,0.0,0.0,0.0,600.0,0.0,2425.30,600.0,29703.60


In [19]:
%%time

idx_last_month_per_employee = airhsp_data.groupby('NUMERO_DOCUMENTO_A')['PERIODO'].idxmax()
last_month_per_employee = airhsp_data.loc[idx_last_month_per_employee]

CPU times: user 2min 41s, sys: 1.58 s, total: 2min 43s
Wall time: 2min 45s


In [20]:
idx_last_month_per_employee.shape

(2406983,)

In [23]:
last_month_per_employee.shape

(2406983, 33)

In [24]:
master_workers = last_month_per_employee[["NUMERO_DOCUMENTO_A","FECHA_INGRESO","FECHA_CESE","DESC_GRUPO_OCUPACIONAL","CENT_RUC","TIPO_REGISTRO"]]

In [28]:
master_workers.shape

(2406983, 7)

In [29]:
master_workers[master_workers['NUMERO_DOCUMENTO_A']==13486841]

,NUMERO_DOCUMENTO_A,FECHA_INGRESO,FECHA_CESE,DESC_GRUPO_OCUPACIONAL,CENT_RUC,TIPO_REGISTRO,ESTADO
26104763,13486841,NaN,NaN,Sin Grupo Ocupacional,20254165035,2,1


In [30]:
%%time
# ESTADO DEL NUMERO_DOCUMENTO_A (activo, no activo)
master_workers["ESTADO"] = master_workers["FECHA_CESE"].apply(lambda x: 1 if pd.isna(x) == True else 0 )

CPU times: user 1.78 s, sys: 0 ns, total: 1.78 s
Wall time: 1.81 s


<timed exec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [31]:
master_workers.shape

(2406983, 7)

In [33]:
%%time
# desviacion estandar del ingreso desglosado del trabajador
worker_ingresos = airhsp_data.groupby(["NUMERO_DOCUMENTO_A"])[["COSTO_PRESUP","TOTAL_MENSUAL","TOTAL_ANUAL"]].std()
worker_ingresos.head(2)

CPU times: user 17.5 s, sys: 226 ms, total: 17.7 s
Wall time: 18 s


,COSTO_PRESUP,TOTAL_MENSUAL,TOTAL_ANUAL
NUMERO_DOCUMENTO_A,,,
10000001,0.0,0.0,0.0
10000002,0.0,0.0,0.0


In [34]:
worker_ingresos.shape

(2406983, 3)

In [35]:
master_workers.shape

(2406983, 7)

In [36]:
master_workers.to_csv(folder_data + "master_worker_part1.csv",index=False)

In [37]:
%%time
master_workers = pd.merge(master_workers, worker_ingresos, on='NUMERO_DOCUMENTO_A', how='left')

CPU times: user 1.56 s, sys: 453 ms, total: 2.02 s
Wall time: 2.05 s


In [62]:
master_workers.shape

(2406983, 10)

In [60]:
# %%time
# # para saber en cuantas entidades ha trabajado este NUMERO_DOCUMENTO_A
# # employee_data["ID"] = employee_data["NUMERO_DOCUMENTO_A"].astype(str) + "-" + employee_data["PERIODO"]
# airhsp_data["ID_2"] = airhsp_data["NUMERO_DOCUMENTO_A"].astype(str) + "-" + airhsp_data["CENT_RUC"].astype(str)

# nro_entidades_table = airhsp_data.groupby(["NUMERO_DOCUMENTO_A","CENT_RUC"])["ID_2"].nunique()
# nro_entidades_df = nro_entidades_table.to_frame('NRO_ENTIDADES').reset_index()
# nro_entidades_df = nro_entidades_df[["NUMERO_DOCUMENTO_A","NRO_ENTIDADES"]]
# nro_entidades_df.head()

In [63]:
%%time

airhsp_data_trab_enti = airhsp_data[["NUMERO_DOCUMENTO_A","CENT_RUC"]].drop_duplicates()
airhsp_data_trab_enti.shape

CPU times: user 10.7 s, sys: 486 ms, total: 11.2 s
Wall time: 11.3 s


(2508038, 2)

In [64]:
conteo_entidades = airhsp_data_trab_enti.groupby(["NUMERO_DOCUMENTO_A"])["CENT_RUC"].count()
conteo_entidades_df = conteo_entidades.to_frame('NRO_ENTIDADES').reset_index()

(2406983, 2)

In [65]:
conteo_entidades_df.shape

(2406983, 2)

In [67]:
%%time
master_workers = pd.merge(master_workers, conteo_entidades_df, on='NUMERO_DOCUMENTO_A', how='left')

CPU times: user 1.52 s, sys: 391 ms, total: 1.91 s
Wall time: 1.93 s


In [68]:
master_workers.shape

(2406983, 11)

In [69]:
master_workers.head()

,NUMERO_DOCUMENTO_A,FECHA_INGRESO,FECHA_CESE,DESC_GRUPO_OCUPACIONAL,CENT_RUC,TIPO_REGISTRO,ESTADO,COSTO_PRESUP,TOTAL_MENSUAL,TOTAL_ANUAL,NRO_ENTIDADES
0,10000001,01/01/1901,NaN,Tecnicos,20145686548,3,1,0.000000,0.000000,0.000000,1
1,10000002,NaN,NaN,Sin Grupo Ocupacional,20254165035,2,1,0.000000,0.000000,0.000000,1
2,10000006,14/03/2019,NaN,Reforma Magisterial,20393274655,1,1,60.332465,63.860151,766.321816,1
3,10000018,01/03/2022,NaN,Auxiliar de Educación,20393274655,1,1,16.410558,4.209729,146.429638,1
4,10000023,14/03/2019,NaN,Profesionales,20393274655,1,1,57.735027,57.735027,692.820323,1


In [70]:
%%time
master_workers.to_csv(folder_data + "master_workers_final.csv", index=False)

CPU times: user 13.3 s, sys: 450 ms, total: 13.7 s
Wall time: 14 s


In [71]:
master_workers.shape

(2406983, 11)

##### ENTIDAD

In [72]:
master_entidad = pd.DataFrame()

In [74]:
airhsp_data.shape

(27639016, 34)

In [75]:
entidad_data = airhsp_data.copy()

In [73]:
# entidad_data = employee_data[(employee_data["CENT_RUC"]==20198261476) | (employee_data["CENT_RUC"]==20195970751)]
# entidad_data.head()

In [76]:
%%time
# datos demograficos por entidad

master_entidad = entidad_data[["CENT_RUC","CENT_NOMBRE","NOMBRE_DEPARTAMENTO","NOMBRE_PROVINCIA","NOMBRE_DISTRITO","OBANT_SECTOR"]].drop_duplicates()

CPU times: user 10.7 s, sys: 2.34 s, total: 13.1 s
Wall time: 13.3 s


In [77]:
master_entidad["CENT_RUC"].shape

(892,)

In [321]:
# # datos demograficos por entidad
# entidad_data["DEMOG_UNIQUE"] = entidad_data["CENT_RUC"].astype(str)+"_"+entidad_data["NOMBRE_DEPARTAMENTO"]+"_"+entidad_data["NOMBRE_PROVINCIA"]+"_"+entidad_data["NOMBRE_DISTRITO"]
# entidad_data_demo_unique = entidad_data["DEMOG_UNIQUE"].nunique()
# # entidad_data_demo_unique["CENT_RUC","NOMBRE_DEPARTAMENTO","NOMBRE_PROVINCIA","NOMBRE_DISTRITO"] = entidad_data_demo_unique["DEMOG_UNIQUE"].apply(lambda x: x.str.split("_"))
# # entidad_data_demo_unique                                                                                                                        

In [79]:
%%time
#nro trabajadores unicos por entidad
nro_trabajadores_table = entidad_data.groupby(["CENT_RUC"])["NUMERO_DOCUMENTO_A"].nunique()
nro_trabajadores_df = nro_trabajadores_table.to_frame('NRO_TRABAJADORES').reset_index()
nro_trabajadores_df.head()

CPU times: user 13.9 s, sys: 572 ms, total: 14.5 s
Wall time: 14.7 s


,CENT_RUC,NRO_TRABAJADORES
0,20105685875,1692
1,20112919377,561
2,20114050530,112
3,20131016396,403
4,20131023414,1591


In [81]:
nro_trabajadores_df.shape

(892, 2)

In [82]:
%%time
master_entidad = pd.merge(master_entidad, nro_trabajadores_df, on='CENT_RUC', how='left')
master_entidad.head()

CPU times: user 3.75 ms, sys: 0 ns, total: 3.75 ms
Wall time: 3.86 ms


,CENT_RUC,CENT_NOMBRE,NOMBRE_DEPARTAMENTO,NOMBRE_PROVINCIA,NOMBRE_DISTRITO,OBANT_SECTOR,NRO_TRABAJADORES
0,20393274655,UNIDAD DE GESTIÓN EDUCATIVA LOCAL CORONEL PORT...,UCAYALI,CORONEL PORTILLO,CALLARIA,GOBIERNOS REGIONALES,10078
1,20195970751,DIRECCIÓN REGIONAL DE EDUCACIÓN UCAYALI,UCAYALI,CORONEL PORTILLO,CALLARIA,GOBIERNOS REGIONALES,591
2,20198261476,DIRECCIÓN REGIONAL DE SALUD UCAYALI,UCAYALI,CORONEL PORTILLO,CALLARIA,GOBIERNOS REGIONALES,1886
3,20254165035,OFICINA DE NORMALIZACIÓN PREVISIONAL - ONP,LIMA,LIMA,LIMA,ECONOMÍA Y FINANZAS,711179
4,20393146657,UNIVERSIDAD NACIONAL INTERCULTURAL DE LA AMAZONIA,UCAYALI,CORONEL PORTILLO,YARINACOCHA,EDUCACIÓN,278


In [83]:
master_entidad.shape

(892, 7)

In [84]:
%%time
#nro trabajadores activos
trabaj_activos_df = master_workers[master_workers["ESTADO"]==1].groupby(["CENT_RUC"])["NUMERO_DOCUMENTO_A"].nunique().to_frame('NRO_TRAB_ACTIVOS').reset_index()

CPU times: user 1.19 s, sys: 195 ms, total: 1.39 s
Wall time: 1.41 s


In [86]:
master_entidad = pd.merge(master_entidad, trabaj_activos_df, on='CENT_RUC', how='left')
master_entidad.head()

,CENT_RUC,CENT_NOMBRE,NOMBRE_DEPARTAMENTO,NOMBRE_PROVINCIA,NOMBRE_DISTRITO,OBANT_SECTOR,NRO_TRABAJADORES,NRO_TRAB_ACTIVOS
0,20393274655,UNIDAD DE GESTIÓN EDUCATIVA LOCAL CORONEL PORT...,UCAYALI,CORONEL PORTILLO,CALLARIA,GOBIERNOS REGIONALES,10078,9183
1,20195970751,DIRECCIÓN REGIONAL DE EDUCACIÓN UCAYALI,UCAYALI,CORONEL PORTILLO,CALLARIA,GOBIERNOS REGIONALES,591,515
2,20198261476,DIRECCIÓN REGIONAL DE SALUD UCAYALI,UCAYALI,CORONEL PORTILLO,CALLARIA,GOBIERNOS REGIONALES,1886,1752
3,20254165035,OFICINA DE NORMALIZACIÓN PREVISIONAL - ONP,LIMA,LIMA,LIMA,ECONOMÍA Y FINANZAS,711179,707010
4,20393146657,UNIVERSIDAD NACIONAL INTERCULTURAL DE LA AMAZONIA,UCAYALI,CORONEL PORTILLO,YARINACOCHA,EDUCACIÓN,278,224


In [87]:
%%time
#nro trabajadores no activos
trabaj_no_activos_df = master_workers[master_workers["ESTADO"]==0].groupby(["CENT_RUC"])["NUMERO_DOCUMENTO_A"].nunique().to_frame('NRO_TRAB_NO_ACTIVOS').reset_index()

CPU times: user 40.6 ms, sys: 87 µs, total: 40.7 ms
Wall time: 42.6 ms


In [88]:
master_entidad = pd.merge(master_entidad, trabaj_no_activos_df, on='CENT_RUC', how='left')
master_entidad.head()

,CENT_RUC,CENT_NOMBRE,NOMBRE_DEPARTAMENTO,NOMBRE_PROVINCIA,NOMBRE_DISTRITO,OBANT_SECTOR,NRO_TRABAJADORES,NRO_TRAB_ACTIVOS,NRO_TRAB_NO_ACTIVOS
0,20393274655,UNIDAD DE GESTIÓN EDUCATIVA LOCAL CORONEL PORT...,UCAYALI,CORONEL PORTILLO,CALLARIA,GOBIERNOS REGIONALES,10078,9183,468.0
1,20195970751,DIRECCIÓN REGIONAL DE EDUCACIÓN UCAYALI,UCAYALI,CORONEL PORTILLO,CALLARIA,GOBIERNOS REGIONALES,591,515,12.0
2,20198261476,DIRECCIÓN REGIONAL DE SALUD UCAYALI,UCAYALI,CORONEL PORTILLO,CALLARIA,GOBIERNOS REGIONALES,1886,1752,47.0
3,20254165035,OFICINA DE NORMALIZACIÓN PREVISIONAL - ONP,LIMA,LIMA,LIMA,ECONOMÍA Y FINANZAS,711179,707010,290.0
4,20393146657,UNIVERSIDAD NACIONAL INTERCULTURAL DE LA AMAZONIA,UCAYALI,CORONEL PORTILLO,YARINACOCHA,EDUCACIÓN,278,224,NaN


In [96]:
%%time
# desviacion estandar del gasto de la entidad por sus trabajadores
entidad_gastos_planilla = entidad_data.groupby(["CENT_RUC"])[["COSTO_PRESUP","TOTAL_MENSUAL","TOTAL_ANUAL"]].agg({
    'COSTO_PRESUP':'sum',
    'TOTAL_MENSUAL':'sum'
})
entidad_gastos_planilla.head()

CPU times: user 584 ms, sys: 111 ms, total: 694 ms
Wall time: 697 ms


,COSTO_PRESUP,TOTAL_MENSUAL
CENT_RUC,,
20105685875,70293892.32,73397057.24
20112919377,28497718.72,27751792.72
20114050530,5560978.26,5006884.51
20131016396,31112582.15,29865970.67
20131023414,65504230.03,66046710.40


In [101]:
entidad_gastos_planilla.shape

(892, 2)

In [102]:
master_entidad = pd.merge(master_entidad, entidad_gastos_planilla, on='CENT_RUC', how='left')
master_entidad.shape

(892, 14)

In [103]:
entidad_gastos_planilla_2 = entidad_data.groupby(["CENT_RUC","NUMERO_DOCUMENTO_A"])[["TOTAL_ANUAL"]].mean()
entidad_gastos = entidad_gastos_planilla_2.groupby("CENT_RUC")["TOTAL_ANUAL"].sum()
entidad_gastos_df = entidad_gastos.to_frame('TOTAL_ANUAL').reset_index()
entidad_gastos_df.head()

,CENT_RUC,TOTAL_ANUAL
0,20105685875,7.969799e+07
1,20112919377,3.128049e+07
2,20114050530,6.416432e+06
3,20131016396,3.471544e+07
4,20131023414,8.106737e+07


In [104]:
master_entidad = pd.merge(master_entidad, entidad_gastos_df, on='CENT_RUC', how='left')
master_entidad.head()

,CENT_RUC,CENT_NOMBRE,NOMBRE_DEPARTAMENTO,NOMBRE_PROVINCIA,NOMBRE_DISTRITO,OBANT_SECTOR,NRO_TRABAJADORES,NRO_TRAB_ACTIVOS,NRO_TRAB_NO_ACTIVOS,COSTO_PRESUP_x,TOTAL_MENSUAL_x,TOTAL_ANUAL_x,COSTO_PRESUP_y,TOTAL_MENSUAL_y,TOTAL_ANUAL_y
0,20393274655,UNIDAD DE GESTIÓN EDUCATIVA LOCAL CORONEL PORT...,UCAYALI,CORONEL PORTILLO,CALLARIA,GOBIERNOS REGIONALES,10078,9183,468.0,2.939537e+08,3.070530e+08,33358.070106,2.939537e+08,3.070530e+08,3.333983e+08
1,20195970751,DIRECCIÓN REGIONAL DE EDUCACIÓN UCAYALI,UCAYALI,CORONEL PORTILLO,CALLARIA,GOBIERNOS REGIONALES,591,515,12.0,1.546762e+07,1.595785e+07,32358.983231,1.546762e+07,1.595785e+07,1.986807e+07
2,20198261476,DIRECCIÓN REGIONAL DE SALUD UCAYALI,UCAYALI,CORONEL PORTILLO,CALLARIA,GOBIERNOS REGIONALES,1886,1752,47.0,5.704301e+07,5.893090e+07,41371.351329,5.704301e+07,5.893090e+07,7.791353e+07
3,20254165035,OFICINA DE NORMALIZACIÓN PREVISIONAL - ONP,LIMA,LIMA,LIMA,ECONOMÍA Y FINANZAS,711179,707010,290.0,5.723522e+09,5.274418e+09,8128.281273,5.723522e+09,5.274418e+09,5.731893e+09
4,20393146657,UNIVERSIDAD NACIONAL INTERCULTURAL DE LA AMAZONIA,UCAYALI,CORONEL PORTILLO,YARINACOCHA,EDUCACIÓN,278,224,NaN,1.034637e+07,1.088394e+07,50398.748509,1.034637e+07,1.088394e+07,1.446251e+07


In [105]:
master_entidad.shape

(892, 15)

In [106]:
master_entidad.to_csv(folder_data + "master_entidad.csv", index=False)

In [ ]:

# FUNCIONARIO
# ATRIBUTO: trabajo_2022 (boolean)
# ATRIBUTO: cese_2022 (boolean)
# ATRIBUTO: activo (boolean)


# ENTIDAD
# ATRIBUTO: nro trabajadores activos
# ATRIBUTO: nro trabajadores que cesaron
# ATRIBUTO: nro trabajadores que trabajaron el 2022

# ATRIBUTO: numero de trabajadores activos según grupo funcional
# ATRIBUTO: numero de trabajadores activos según TIPO-REGISTRO
# ATRIBUTO: suma de costo presupuestado
# ATRIBUTO: promedio de incentivo unico
# ATRIBUTO: promedio del total mensual
# ATRIBUTO: promedio del total anual

desviaci[on estandar

# normalizar DESC_GRUPO_OCUPACIONAL



In [61]:
custom_airhsp = airhsp_data[["NUMERO_DOCUMENTO_A","DESC_GRUPO_OCUPACIONAL","CENT_RUC","CENT_NOMBRE","OBANT_GRUPO_FUNCIONAL","COSTO_PRESUP","INCENTIVO_UNICO","TOTAL_MENSUAL"]]

In [62]:
custom_airhsp.head()

,NUMERO_DOCUMENTO_A,DESC_GRUPO_OCUPACIONAL,CENT_RUC,CENT_NOMBRE,OBANT_GRUPO_FUNCIONAL,COSTO_PRESUP,INCENTIVO_UNICO,TOTAL_MENSUAL
0,10000482,Profesorado,20393274655,UNIDAD DE GESTIÓN EDUCATIVA LOCAL CORONEL PORT...,GOBIERNO REGIONAL,1267.68,0.0,1184.35
1,10002552,Profesionales,20195970751,DIRECCIÓN REGIONAL DE EDUCACIÓN UCAYALI,GOBIERNO REGIONAL,2262.97,1230.0,2263.34
2,10005000,Tecnicos,20198261476,DIRECCIÓN REGIONAL DE SALUD UCAYALI,GOBIERNO REGIONAL,3211.88,0.0,3252.98
3,10008979,Reforma Magisterial,20393274655,UNIDAD DE GESTIÓN EDUCATIVA LOCAL CORONEL PORT...,GOBIERNO REGIONAL,2583.63,0.0,2740.72
4,10011497,Sin Grupo Ocupacional,20254165035,OFICINA DE NORMALIZACIÓN PREVISIONAL - ONP,ECONOMICA PRODUCTIVA,385.00,0.0,385.00


In [63]:
custom_airhsp.to_csv(folder_data + "airhsp_v1.csv", index=False)

### SEACE

In [49]:
%%time

seace_data = get_dataset('ds_seace',0)


Tamaños de archivos CSV en DS02_SEACE.zip:
['DS02_SEACE.csv']
DS02_SEACE.csv
CPU times: user 1.86 s, sys: 56.9 ms, total: 1.92 s
Wall time: 1.94 s


In [62]:
seace_data.shape

(248253, 24)

In [50]:
seace_data.shape

(248253, 19)

In [51]:
len(seace_data["CENT_RUC"].unique())

3004

In [52]:
seace_data.head(5)

,N_ID_EXPEDE,CENT_RUC,CENT_NOMBRE,OBANT_NIVEL_GOB,OBANT_SECTOR,NOMBRE_DEPARTAMENTO,NOMBRE_PROVINCIA,NOMBRE_DISTRITO,TIPO_PROCESO,PROCESO_SELECCION,FECHA_PUBLICACION,FECHA_ADJUDICACION,TIPO_MONEDA,TIPO_CAMBIO_CONV,MONTO_ADJUDICADO,MONTO_ADJ_PROVEEDOR,NUMERO_ITEMS,RUC_PROVEEDOR,ES_CONSORCIO
0,680584,20555195444,SUPERINTENDENCIA NACIONAL DE FISCALIZACIÓN LAB...,GOBIERNO NACIONAL,TRABAJO Y PROMOCIÓN DEL EMPLEO,LIMA,LIMA,JESUS MARIA,Contratación Directa,DIRECTA-PROC-8-2020-SUNAFIL-1,15/12/2020,16/12/2020,S/,1.0,204000.00,204000.000,1,10101723050,N
1,653921,20602398103,REGION POLICIAL HUANUCO - SAN MARTIN - UCAYALI,GOBIERNO NACIONAL,INTERIOR,HUANUCO,HUANUCO,HUANUCO,Adjudicación Simplificada,AS-SM-6-2020-UEHSMU-4,29/09/2020,13/10/2020,S/,1.0,224913.00,224913.000,1,10102123960,N
2,590946,20207736261,PROYECTO ESPECIAL BINACIONAL DESARROLLO INTEGR...,GOBIERNO NACIONAL,AGRARIO Y DE RIEGO,LORETO,MAYNAS,IQUITOS,Adjudicación Simplificada,AS-SM-16-2019-PEBDICP/OEC-1,24/12/2019,30/01/2020,S/,1.0,72000.00,72000.000,1,10102395900,N
3,592653,20548776920,PROGRAMA NACIONAL DE SANEAMIENTO RURAL,GOBIERNO NACIONAL,VIVIENDA CONSTRUCCIÓN Y SANEAMIENTO,LIMA,LIMA,MIRAFLORES,Adjudicación Simplificada,AS-SM-71-2019-PNSR-1,30/12/2019,17/02/2020,S/,1.0,129756.44,77853.864,1,10102560950,S
4,633502,20148170933,MUNICIPALIDAD PROVINCIAL DE RIOJA,GOBIERNO LOCAL,GOBIERNO LOCAL,SAN MARTIN,RIOJA,RIOJA,Regímen Especial,RES-PROC-12-2020-MPR/CS-1,31/07/2020,14/08/2020,S/,1.0,65299.20,65299.200,1,10102560950,N


In [53]:
# normalizar la unidad monetaria (convertir todos los montos a soles)
seace_data["MONTO_ADJUDICADO_SOLES"] = seace_data["MONTO_ADJUDICADO"] * seace_data["TIPO_CAMBIO_CONV"]
seace_data["MONTO_ADJ_PROVEEDOR_SOLES"] = seace_data["MONTO_ADJ_PROVEEDOR"] * seace_data["TIPO_CAMBIO_CONV"]

In [54]:
# Asegúrate de que las fechas están en el formato correcto
seace_data['FECHA_PUBLICACION'] = pd.to_datetime(seace_data['FECHA_PUBLICACION'])
seace_data['FECHA_ADJUDICACION'] = pd.to_datetime(seace_data['FECHA_ADJUDICACION'])
seace_data['ANO_ADJ'] = seace_data['FECHA_ADJUDICACION'].dt.year

# Preparar DataFrame para cálculos
grouped = seace_data.groupby(['CENT_RUC', 'ANO_ADJ'])

# Contar valores únicos y sumas
nuevo_df = grouped.agg(
    CANTI_EXP=('N_ID_EXPEDE', 'nunique'),
    TOTAL_ADJ_PROVEEDOR_SOLES=('MONTO_ADJ_PROVEEDOR_SOLES', 'sum'),
    NRO_PROVEEDORES=('RUC_PROVEEDOR', 'nunique')
)

# Calcular porcentajes para cada tipo de proceso
tipos_proceso = ['Contratación Directa', 'Adjudicación Simplificada', 'Subasta Inversa Electrónica', 
                 'Licitación Pública', 'Regímen Especial', 'Concurso Público', 'Procedimiento Especial de Contratación']
for tipo in tipos_proceso:
    nuevo_df[f'PORC_{tipo.replace(" ", "_").upper()}'] = (
        seace_data[seace_data['TIPO_PROCESO'] == tipo].groupby(['CENT_RUC', 'ANO_ADJ'])['N_ID_EXPEDE'].count()
        / nuevo_df['CANTI_EXP']
    ).fillna(0) * 100

# Diferencia en días y su desviación estándar
seace_data['DIF_PUB_ADJU'] = (seace_data['FECHA_ADJUDICACION'] - seace_data['FECHA_PUBLICACION']).dt.days
nuevo_df['STD_DIF_PUB_ADJU'] = grouped['DIF_PUB_ADJU'].std()

# Calcular porcentaje de adjudicación a proveedor y su desviación estándar
seace_data['PORC_ADJ_PROVEEDOR'] = (seace_data['MONTO_ADJ_PROVEEDOR_SOLES'] / seace_data['MONTO_ADJUDICADO_SOLES']) * 100
nuevo_df['STD_PORC_ADJ_PROVEEDOR'] = grouped['PORC_ADJ_PROVEEDOR'].std()

# Calcular porcentaje de proveedores en consorcio
nuevo_df['PORC_PROVEEDOR_CONS'] = (
    seace_data[seace_data['ES_CONSORCIO'] == 'S'].groupby(['CENT_RUC', 'ANO_ADJ'])['RUC_PROVEEDOR'].count()
    / nuevo_df['NRO_PROVEEDORES']
).fillna(0) * 100

# Restaurar índices y agregar columnas faltantes
nuevo_df.reset_index(inplace=True)
nuevo_df = nuevo_df.merge(
    seace_data[['CENT_RUC', 'CENT_NOMBRE', 'NOMBRE_DEPARTAMENTO', 'NOMBRE_PROVINCIA', 'NOMBRE_DISTRITO']].drop_duplicates(),
    on='CENT_RUC',
    how='left'
)


/tmp/ipykernel_1572254/1334909893.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  seace_data['FECHA_PUBLICACION'] = pd.to_datetime(seace_data['FECHA_PUBLICACION'])
/tmp/ipykernel_1572254/1334909893.py:3: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  seace_data['FECHA_ADJUDICACION'] = pd.to_datetime(seace_data['FECHA_ADJUDICACION'])


In [55]:
nuevo_df.shape

(8331, 19)

In [56]:
nuevo_df.head()

,CENT_RUC,ANO_ADJ,CANTI_EXP,TOTAL_ADJ_PROVEEDOR_SOLES,NRO_PROVEEDORES,PORC_CONTRATACIÓN_DIRECTA,PORC_ADJUDICACIÓN_SIMPLIFICADA,PORC_SUBASTA_INVERSA_ELECTRÓNICA,PORC_LICITACIÓN_PÚBLICA,PORC_REGÍMEN_ESPECIAL,PORC_CONCURSO_PÚBLICO,PORC_PROCEDIMIENTO_ESPECIAL_DE_CONTRATACIÓN,STD_DIF_PUB_ADJU,STD_PORC_ADJ_PROVEEDOR,PORC_PROVEEDOR_CONS,CENT_NOMBRE,NOMBRE_DEPARTAMENTO,NOMBRE_PROVINCIA,NOMBRE_DISTRITO
0,20100003199,2020.0,19,8.906413e+06,19,5.263158,63.157895,0.0,5.263158,0.0,26.315789,0.0,26.206618,15.389675,10.526316,EMPRESA NACIONAL DE PUERTOS S.A.,CALLAO,CALLAO,CALLAO
1,20100003199,2021.0,19,7.326682e+06,18,5.263158,63.157895,0.0,5.263158,0.0,26.315789,0.0,19.869898,0.000000,0.000000,EMPRESA NACIONAL DE PUERTOS S.A.,CALLAO,CALLAO,CALLAO
2,20100003199,2022.0,19,6.527610e+06,22,0.000000,110.526316,0.0,0.000000,0.0,15.789474,0.0,21.412063,21.802805,27.272727,EMPRESA NACIONAL DE PUERTOS S.A.,CALLAO,CALLAO,CALLAO
3,20100003351,2020.0,32,1.459707e+07,36,15.625000,93.750000,0.0,3.125000,0.0,18.750000,0.0,17.404727,30.336556,38.888889,SERVICIOS INDUSTRIALES DE LA MARINA S.A.,CALLAO,CALLAO,CALLAO
4,20100003351,2021.0,44,2.028464e+07,42,2.272727,81.818182,0.0,4.545455,0.0,25.000000,0.0,14.355899,22.768578,16.666667,SERVICIOS INDUSTRIALES DE LA MARINA S.A.,CALLAO,CALLAO,CALLAO


In [57]:
nuevo_df.to_csv(folder_data + "master_entidad_seace.csv", index=False)

### INCO

In [60]:
def get_dataset(id_dataset,id_elem):
    archivo_zip = datasets[id_dataset]
    ruta_zip = os.path.join(folder_data, archivo_zip)
    df = pl.DataFrame()
    
    with zipfile.ZipFile(ruta_zip, 'r') as zip_ref:
        print(f"\nTamaños de archivos CSV en {archivo_zip}:")
        # Iterar sobre los archivos dentro del zip
        list_archivo = []
        for nombre_archivo in zip_ref.namelist():
            # Verificar si el archivo es un archivo CSV
            
            if nombre_archivo.endswith('.xlsx'):
                list_archivo.append(nombre_archivo)
            
        print(list_archivo)
        print(list_archivo[id_elem])
        with zip_ref.open(list_archivo[id_elem]) as archivo_csv:
            # df = pl.read_csv(archivo_csv, ignore_errors=True)
            df = pd.read_csv(archivo_csv)
            
        # Obtener el tamaño del archivo en bytes
        # tamano_bytes = zip_ref.getinfo(nombre_archivo).file_size

        # # Convertir el tamaño a megabytes para mayor legibilidad
        # tamano_mb = tamano_bytes / (1024 ** 3)

        # # Imprimir el nombre del archivo y su tamaño
        # print(f'{nombre_archivo}: {tamano_mb:.2f} GB')
            # break
    return df

In [61]:
%%time

inco_data = get_dataset('ds_inco',0)


Tamaños de archivos CSV en DS08_INCO.zip:
['08_DS_INCO/Resultados_INCO_2022_TABS AGRUPADOS_PORTAL WEB.xlsx', '08_DS_INCO/Resultados Finales_INCO 2021.xlsx', '08_DS_INCO/Resultados Finales_INCO 2022.xlsx', '08_DS_INCO/Resultados_INCO_2022_FICHAS ENTIDADES_PORTAL WEB.xlsx']
08_DS_INCO/Resultados_INCO_2022_TABS AGRUPADOS_PORTAL WEB.xlsx


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xfb in position 15: invalid start byte